## Imports

In [3]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup as BS
import cloudscraper
import os.path
import json

## Get airports and their connecting airports

In [4]:
airportsFile = open("airportList.csv")
airports = airportsFile.readlines()
airports = airports[0].split(',')

## Get connecting airports

In [5]:
# We want to webscrape urls that look like this:
# https://www.flightsfrom.com/ATL/destinations?countries=US&durationFrom=44&durationTo=999
scraper = cloudscraper.create_scraper()
connections = {}
if not os.path.isfile("utils/outgoingDictionary.txt"):
    for airport in airports:
        url = "https://www.flightsfrom.com/" + airport + "/destinations?countries=US&durationFrom=44&durationTo=999"
        r = scraper.get(url)
        soup = BS(r.text, 'html.parser')
        outgoingAirports = soup.find_all(class_ = "airport-font-midheader destination-search-item")
        connections[airport] = []
        for outgoing in outgoingAirports:
            connections[airport].append(str(outgoing).split(" ")[-1][:3])
    f = open("utils/outgoingDictionary.txt","x")
    json.dump(connections,f)
else:
    connections = json.load(open("utils/outgoingDictionary.txt"))


## Create links to all flight searches

In [20]:
#We want to create urls that look like this:
# "https://www.google.com/travel/flights?q=one%20way%20Flights%20to%20SNA%20from%20ATL%20on%202023-09-13"
if not os.path.isfile("flightLinks.csv"):
    f = open("flightLinks.csv","x")
    for startAirport in connections.keys():
        for endAirport in connections[startAirport]:
            searchURL = "https://www.google.com/travel/flights?q=one%20way%20Flights%20to%20"+endAirport+"%20from%20"+startAirport+"%20on%20,"
            f.write(searchURL+"\n")
    f.close()

## Create Airport Integer Encodings

In [ ]:
airport_to_int = {}
for i,airport in enumerate(airports):
    airport_to_int[airport] = i
